In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns

In [2]:
dataset = pd.read_excel('/Users/marjorylamothe/Downloads/Projet Get Around/get_around_delay_analysis.xlsx')

In [3]:
print("Number of rows : {}".format(dataset.shape[0]))
print()

print("Display of dataset: ")
display(dataset.head())
print()

print("Basics statistics: ")
data_apr14_desc = dataset.describe(include='all')
display(data_apr14_desc)
print()

print("Percentage of missing values: ")
display(100*dataset.isnull().sum()/dataset.shape[0])

Number of rows : 21310

Display of dataset: 


,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes
0,505000,363965,mobile,canceled,NaN,NaN,NaN
1,507750,269550,mobile,ended,-81.0,NaN,NaN
2,508131,359049,connect,ended,70.0,NaN,NaN
3,508865,299063,connect,canceled,NaN,NaN,NaN
4,511440,313932,mobile,ended,NaN,NaN,NaN



Basics statistics: 


,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes
count,21310.000000,21310.000000,21310,21310,16346.000000,1841.000000,1841.000000
unique,NaN,NaN,2,2,NaN,NaN,NaN
top,NaN,NaN,mobile,ended,NaN,NaN,NaN
freq,NaN,NaN,17003,18045,NaN,NaN,NaN
mean,549712.880338,350030.603426,NaN,NaN,59.701517,550127.411733,279.288430
std,13863.446964,58206.249765,NaN,NaN,1002.561635,13184.023111,254.594486
min,504806.000000,159250.000000,NaN,NaN,-22433.000000,505628.000000,0.000000
25%,540613.250000,317639.000000,NaN,NaN,-36.000000,540896.000000,60.000000
50%,550350.000000,368717.000000,NaN,NaN,9.000000,550567.000000,180.000000
75%,560468.500000,394928.000000,NaN,NaN,67.000000,560823.000000,540.000000



Percentage of missing values: 


rental_id                                      0.000000
car_id                                         0.000000
checkin_type                                   0.000000
state                                          0.000000
delay_at_checkout_in_minutes                  23.294228
previous_ended_rental_id                      91.360863
time_delta_with_previous_rental_in_minutes    91.360863
dtype: float64

In [4]:
dataset['state'].value_counts()

ended       18045
canceled     3265
Name: state, dtype: int64

How many rentals would be affected by the feature depending on the threshold and scope we choose?

In [5]:
dataset["delay_category"] = dataset["delay_at_checkout_in_minutes"].apply(lambda x: "No delay" if x < 0 
                                        else "Less than 15 minutes" if x >= 0 and x < 15
                                        else "15 to 30 minutes" if x >= 15 and x < 30
                                        else "30 minutes to 1 hours" if x >= 30 and x < 60
                                        else "1 to 2 hours" if x >= 60 and x < 120 
                                        else "2 to 3 hours" if x >= 120 and x < 180 
                                        else "3 to 4 hours" if x >= 180 and x < 240 
                                        else "4 to 5 hours" if x >= 240 and x < 300 
                                        else "5 to 6 hours" if x >= 300 and x < 360 
                                        else "6 to 12 hours" if x >= 360 and x < 720 
                                        else "12 hours to a day" if x >= 720 and x < 1440 
                                        else "1 to 5 days" if x >= 1440 and x < 7200
                                        else "5 to 10 days" if x >= 7200 and x < 14400 
                                        else "10 to 15 days" if x >= 14400 and x < 21600 
                                        else "More than 15 days" if x > 21600
                                        else x)

In [6]:
delay=pd.DataFrame(dataset["delay_category"].value_counts())

In [7]:
fig1 = px.bar(delay, y='delay_category', title='Number of rentals depending on possible threshold', color='delay_category')
fig1.show()

How often are drivers late for the next check-in?

In [8]:
late_drivers = 100*len(dataset[dataset['time_delta_with_previous_rental_in_minutes']>0])/dataset.shape[0]

In [9]:
drivers_not_late=100-late_drivers

In [10]:
percentage_late_next_checkin={
    'Category' : ['Late for next check-in', 'Not Late for next chek-in'],
    'Percent' : [late_drivers, drivers_not_late]
}

In [11]:
fig2 = px.pie(percentage_late_next_checkin, values='Percent', names='Category')
fig2.show()

How does it impact the next driver?

In [12]:
state_late=pd.DataFrame(dataset['state'].loc[dataset['delay_category'] != 'No delay'])
cancelation_late=100*len(state_late[state_late['state']=='canceled'])/state_late.shape[0]

In [13]:
late_not_canceled=100-cancelation_late

In [14]:
percentage_cancelation_when_late={
    'Category' : ['Canceled', 'Not canceled'],
    'Percent' : [cancelation_late, late_not_canceled]
}

In [15]:
state_not_late=pd.DataFrame(dataset['state'].loc[dataset['delay_category'] == 'No delay'])
cancelation_not_late=100*len(state_not_late[state_not_late['state']=='canceled'])/state_not_late.shape[0]

In [16]:
not_late_not_canceled=100-cancelation_not_late

99.98533724340176

In [17]:
percentage_cancelation_when_not_late={
    'Category' : ['Canceled', 'Not canceled'],
    'Percent' : [cancelation_not_late, not_late_not_canceled]
}

In [18]:
fig3 = px.pie(percentage_cancelation_when_late, values='Percent', names='Category')
fig3.show()

In [19]:
fig4 = px.pie(percentage_cancelation_when_not_late, values='Percent', names='Category')
fig4.show()

How many problematic cases will it solve depending on the chosen threshold and scope?
